In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [17]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:
            key = list(chunk.keys())[0]
            if key == 'agent':
                content = chunk['agent']['messages'][0].content if chunk['agent']['messages'][0].content != '' else chunk['agent']['messages'][0].additional_kwargs
                print('agent:', content)
            elif key == 'tools':
                for tool_msg in chunk['tools']['messages']:
                    print('tools:', tool_msg.content)
            results.append(chunk)
        return results
    except Exception as e:
        print('Error processing stream:', e)
        return results
    

In [18]:

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# As of langchain-mcp-adapters 0.1.0, MultiServerMCPClient cannot be used this way
# async with MultiServerMCPClient(
#     {
#         "math": {
#             "command": "python",
#             "args": ["./mcp_server.py"],
#             "transport": "stdio",
#         }
#     }
# ) as client:
#     tool_list = client.get_tools()
#     agent = create_react_agent(llm, tool_list)
#     query = "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?"
#     system_prompt = client.get_prompt(server_name="house_tax", prompt_name="house_tax_system_prompt", arguments={})
#     messages = await system_prompt + [HumanMessage(content=query)]
#     stream_generator = agent.astream({"messages": messages})

#     all_chunks = await process_stream(stream_generator)

#     if all_chunks:
#         final_result = all_chunks[-1]
#         print("Final result:", final_result)

In [19]:
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.prompts import load_mcp_prompt
from mcp import StdioServerParameters, ClientSession
from mcp.client.stdio import stdio_client

mcp_server_params = StdioServerParameters(
    command="python",
    args=["./mcp_server.py"]
)
async def run_agent():
    async with stdio_client(mcp_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # init connection
            await session.initialize()

            tool_list = await load_mcp_tools(session)
            agent = create_react_agent(llm, tool_list)
            query = '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'
            system_prompt = await load_mcp_prompt(session, 'house_tax_system_prompt', arguments={})
            messages = system_prompt + [HumanMessage(query)]
            stream_generator = agent.astream({'messages': messages})
            all_chunks = await process_stream(stream_generator)

            if all_chunks:
                final_result = all_chunks[-1]
                print("Final result:", final_result)




In [20]:
import nest_asyncio
nest_asyncio.apply()

In [21]:
import asyncio
asyncio.run(run_agent())

agent: {'tool_calls': [{'id': 'call_wvmYpCUVsi4EHGdbPf6AvMiw', 'function': {'arguments': '{}', 'name': 'tax_base_tool'}, 'type': 'function'}, {'id': 'call_MWcXCtS33D3BIciAwE5EC1Fb', 'function': {'arguments': '{"question": "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채에 대한 세금 공제액은 얼마인가요?"}', 'name': 'tax_deductible_tool'}, 'type': 'function'}, {'id': 'call_8d5zn6RzX1twQRtFdGVEIPRY', 'function': {'arguments': '{"question": "5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채에 대한 공정시장가액비율은 얼마인가요?"}', 'name': 'market_value_rate_tool'}, 'type': 'function'}], 'refusal': None}
tools: 과세표준 = (주택의 공시가격 합산 - 공제금액) × 공정시장가액비율
tools: 답변: 0원
tools: 45%
agent: {'tool_calls': [{'id': 'call_vFyM8T6srxYLa5KNlgNE51HB', 'function': {'arguments': '{"tax_base_question":"과세표준 = (주택의 공시지가 합산 - 공제액) × 공정시장가액비율","market_value_rate":"45%","tax_deductible":"0원","question":"5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채에 대한 종합부동산세는 얼마인가요?"}', 'name': 'house_tax_tool'}, 'type': 'function'}], 'refusal': None}
tools: 주택의 공시지가 합산은 5억 원 + 10억 원 + 20억 원 = 35억 원